In [ ]:
# Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import os

In [ ]:
# Models load
mobilenet_model_path = "/content/drive/My Drive/My_Plant_Proj/Dataset/mobilenetv2_fast.h5"
densenet_model_path = "/content/drive/My Drive/My_Plant_Proj/Dataset/densenet121_fast.h5"
efficientnet_model_path = "/content/drive/My Drive/My_Plant_Proj/Dataset/efficientnetb0_fast.h5"

mobilenet = load_model(mobilenet_model_path)
densenet = load_model(densenet_model_path)
efficientnet = load_model(efficientnet_model_path)

In [ ]:
# Prepare validation data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

IMG_SIZE = (128, 128)
BATCH_SIZE = 64
valid_dir = "/content/drive/My Drive/My_Plant_Proj/Dataset/Dataset/valid"
MAX_PER_CLASS = 180

def get_limited_image_paths(directory, max_per_class=180):
    image_paths = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            all_images = os.listdir(class_path)
            selected_images = all_images[:max_per_class]
            for img in selected_images:
                image_paths.append(os.path.join(class_path, img))
                labels.append(class_name)
    return pd.DataFrame({"filename": image_paths, "class": labels})

valid_df = get_limited_image_paths(valid_dir, MAX_PER_CLASS)

datagen = ImageDataGenerator(rescale=1./255)
valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_df, directory=None, x_col="filename", y_col="class",
    target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

In [ ]:
# Predictions using individual models
y_pred_mobilenet = mobilenet.predict(valid_generator)
y_pred_densenet = densenet.predict(valid_generator)
y_pred_efficientnet = efficientnet.predict(valid_generator)

In [ ]:
# Soft weighted ensemble
weights = [0.3, 0.3, 0.4]  # temp weights (use gradcam to better understand the weights)
ensemble_pred = (weights[0] * y_pred_mobilenet +
                 weights[1] * y_pred_densenet +
                 weights[2] * y_pred_efficientnet)

ensemble_classes = np.argmax(ensemble_pred, axis=1)
true_classes = valid_generator.classes

In [ ]:
# Accuracy
accuracy = accuracy_score(true_classes, ensemble_classes)
print(f"Ensemble Accuracy: {accuracy:.4f}")